In [ ]:
import csv
import random
import math
import operator
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
import nltk
from nltk.corpus import stopwords
import re

random.seed(101)
model = KeyedVectors.load('model.bin')

In [ ]:
#Loading our dataset
def dataload(file,split,validationSpilt,train=[],validation=[],test=[]):
    #print(file)
    with open(file,'r') as csvfile:
        lines=csv.reader(csvfile)
        data=list(lines)
    #data = data.dropna(axis=1, how='any')
    for x in range(1, len(data)-1):
        #print(data[x])
        l=data[x]
        d = []
        for i in l:
            if i != '':
                d.append(i)
        #print(d)
        #break
        if(x<split):
            if(random.random()>validationSpilt):
                validation.append(d)
            else:
                train.append(d)
        else:
            test.append(d)

In [ ]:
def cosineDistance(x, y):
    #print(length)
    #print(len(y))
    distance=0
    magnitude_x=0
    magnitude_y=0
    #print(x[0])
    #print(y)
    product=0
    for i in range(0,len(x)):
        magnitude_x += x[i]*x[i]
    for i in range(0,len(y)):
        magnitude_y +=y[i]*y[i]
    for i in range(0,len(x)):
        product+=x[i]*y[i]
    #print(product)
    #print(magnitude_x)
    #print(magnitude_y)
    distance=float(product)/float((math.sqrt(magnitude_x)*math.sqrt(magnitude_y)))
    return(1-distance)

In [ ]:
def average_vec(sentence):
    words = sentence.split()
    word_vecs = [model[w] for w in words]
    #print(len(word_vecs))    
    #return word_vecs
    return (np.array(word_vecs).sum(axis=0)/len(word_vecs)).reshape(1,-1)

In [ ]:
def cos_sim(a, b):
    """Takes 2 vectors a, b and returns the cosine similarity according 
    to the definition of the dot product
    """
    a = np.asarray(a,dtype=float)
    b = np.asarray(b,dtype=float)
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    cos = dot_product / (norm_a * norm_b)
    return (1-cos)

"""
def cleanup(sentence):
    trainList=[]
    sentence=str(sentence)
    print(type(sentence))
    train_str=sentence.lower()
    words=words=nltk.word_tokenize(train_str)
    for word in words:
        if (word.isalpha() and not word in stopword_set):
            trainList.append(word)
    train_str=' '.join(trainList)
    return train_str"""

In [ ]:
def cleanup(sentence):
    trainList=[]
    #print(type(sentence))
    train_str=sentence.lower()
    words=words=nltk.word_tokenize(train_str)
    for word in words:
        if (not word.isdigit() and not word in stopword_set and re.search('[a-zA-Z]', word)):
            #word.translate(None, string.punctuation)
            trainList.append(word)
    train_str=' '.join(trainList)
    #print(train_str)
    return train_str

def removePunct(word):
    # define punctuation
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

    # To take input from the user
    # my_str = input("Enter a string: ")

    # remove punctuation from the string
    no_punct = ""
    for char in word:
        if char not in punctuations:
            no_punct = no_punct + char
        else:
            no_punct = no_punct + ' '
    return no_punct        

In [ ]:
def fullDistFinder(train,test):
    noOfColumnsTestSet=len(test)
    #print("--------------",noOfColumnsTestSet,"------------------------")
    trainWithoutTarget=train[:(noOfColumnsTestSet)]
    train_num = []
    train_str = []
    test_num = []
    test_str = []

    #print(len(test))
    #print('**********************************************')
    for i in range(0,noOfColumnsTestSet):
        if i in [0,1,2,5]:
            if trainWithoutTarget[i].isdigit() and test[i].isdigit():
                train_num.append(float(trainWithoutTarget[i]))
                test_num.append(float(test[i]))
        else:
            #x=cleanup(train1[i])
            if(not trainWithoutTarget[i].isdigit()):
                #if(word.isalpha() and not word in stopword_set):
                x=cleanup(trainWithoutTarget[i])
                x=removePunct(x)
                train_str.append(x)

            if(not test[i].isdigit()):
                x=cleanup(test[i])
                x=removePunct(x)
                test_str.append(x)

    dist1 = cosineDistance(train_num, test_num)
    trainS = ' '.join(train_str)
    testS = ' '.join(test_str)
    dist2 = cosineDistance(average_vec(trainS)[0], average_vec(testS)[0])
    #print(dist2)
    #return(0)
    dist = (dist1 + dist2)/2
    return dist

In [ ]:
#Returning K nearest neighbours based on distance
def Neighbors(train, test, k):
    distances = []
    length = len(test)
    testWithoutTarget=test[:(length-1)]
    for x in range(len(train)):
        dist = fullDistFinder(train[x],testWithoutTarget)
        distances.append((train[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors


In [ ]:
#Voting the classes obtained from Neighbours
def Response(neighbors):
    Votes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in Votes:
            Votes[response] += 1
        else:
            Votes[response] = 1
    sortedVotes = sorted(Votes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [ ]:
#Calculating Accuracy
def Accuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        cleanedTest=cleanup(testSet[x][-1])
        cleanedTest=removePunct(cleanedTest)
        cleanedPred=cleanup(predictions[x])
        cleanedPred=removePunct(cleanedPred)
        #print(cleanedTest)
        #print(cleanedPred)
        testVec=average_vec(cleanedTest)[0]
        predVec=average_vec(cleanedPred)[0]
        #print(len(testVec))
        #print(len(predVec))
        distance=cosineDistance(testVec,predVec)
        similarity=1-distance
        if similarity>=0.95:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [ ]:
#Combining all functions to get KNN
random.seed(101)
trainingSet=[] #training set contains 11 columns with the last column being the target column
test=[]        #test set contains 10 columns.
validationSet=[]
split = 3000
validationSplit=0.8
dataload('New.csv', split,validationSplit, trainingSet,validationSet, test)
#print(trainingSet)
#print(test)
stopword_set = set(stopwords.words('english'))
trainingSet=[x[1:] for x in trainingSet]
#trainingSet=trainingSet[:100]
validationSet=[x[1:] for x in validationSet]
#validationSet=validationSet[:5]
test=[x[1:] for x in test]
#print(test)
#print('Train set: ',trainingSet[0])
#print('Validation set: ',validationSet[0][5])
#print
#print(len(trainingSet))
#print(len(validationSet))

#k=5
predictions=[]
resultForAllK=[]
predictionsForAllK=[]
accuracyForAllK=[]
for k in range(1,6):
    temp1=[]
    temp2=[]
    predictions=[]
    for x in range(len(validationSet)):
        #print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        neighbors = Neighbors(trainingSet, validationSet[x], k)
        #print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        result = Response(neighbors)
        temp1.append(result)
        predictions.append(result)
        #print('Predicted=',result,', Actual=',validationSet[x][-1])
    resultForAllK.append(temp1)
    predictionsForAllK.append(predictions)
    accuracy =Accuracy(validationSet, predictions)
    accuracyForAllK.append(accuracy)
    print(accuracy)
